## Contextualized model - Using context first

Results when using context first - this is deprecated, now we use the context at the end of our input

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [1]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets

train_dataset, dev_dataset, test_dataset = load_datasets(add_body=True)


In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

context_model_name = "../models/bert-contextualized-hate-speech-es/"
no_context_model_name = "../models/bert-non-contextualized-hate-speech-es"
full_context_model_name = "../models/bert-title-body-hate-speech-es/"


context_model = AutoModelForSequenceClassification.from_pretrained(context_model_name, return_dict=True, num_labels=2)
no_context_model = AutoModelForSequenceClassification.from_pretrained(no_context_model_name, return_dict=True, num_labels=2)
full_context_model = AutoModelForSequenceClassification.from_pretrained(no_context_model_name, return_dict=True, num_labels=2)

context_model.eval()
no_context_model.eval()
full_context_model.eval()


device = "cuda" if torch.cuda.is_available() else "cpu"

context_model = context_model.to(device)
no_context_model = no_context_model.to(device)
full_context_model = full_context_model.to(device)
# Tienen mismo tokenizer así que todo bien

no_context_tokenizer = AutoTokenizer.from_pretrained(no_context_model_name)
context_tokenizer = AutoTokenizer.from_pretrained(context_model_name)
full_context_tokenizer = AutoTokenizer.from_pretrained(context_model_name)

no_context_tokenizer.model_max_length = 128
context_tokenizer.model_max_length = 256

In [9]:
from hatedetection.training import tokenize

batch_size = 32
eval_batch_size = 16

dataset = test_dataset


opts = {"context_first": True}

no_context_dataset = dataset.map(lambda x: tokenize(no_context_tokenizer, x, context='none'), batched=True, batch_size=eval_batch_size)
context_dataset = dataset.map(lambda x: tokenize(context_tokenizer, x, context='title', **opts), batched=True, batch_size=eval_batch_size)
full_context_dataset = dataset.map(lambda x: tokenize(full_context_tokenizer, x, context='title+body', **opts), batched=True, batch_size=eval_batch_size)


In [10]:

def format_dataset(dataset):
    dataset = dataset.map(lambda examples: {'labels': examples['HATEFUL']})
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
    return dataset

no_context_dataset = format_dataset(no_context_dataset)
context_dataset = format_dataset(context_dataset)
full_context_dataset = format_dataset(full_context_dataset)


Sanity check

In [11]:
context_tokenizer.decode(context_dataset[0]["input_ids"])

'[CLS] mora godoy cierra su escuela de tango y remata su vestuario para " poder seguir adelante " [SEP] [USER] ojalá se funda y cague de hambre [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

Bien, primero contexto

Lo cargamos sólo para evaluar 🤗

In [12]:
from hatedetection.metrics import compute_hate_metrics
from transformers import Trainer, TrainingArguments

eval_batch_size = 32

training_args = TrainingArguments(
    output_dir=".",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
)


context_trainer = Trainer(
    model=context_model,
    args=training_args,
    compute_metrics=compute_hate_metrics,
)

full_context_trainer = Trainer(
    model=full_context_model,
    args=training_args,
    compute_metrics=compute_hate_metrics,
)

no_context_trainer = Trainer(
    model=no_context_model,
    args=training_args,
    compute_metrics=compute_hate_metrics,
)


## Resultados

In [13]:
import pandas as pd
pd.options.display.max_columns = 40
pd.set_option('display.float_format', lambda x: '%.5f' % x)


df_full_context_results = pd.DataFrame([full_context_trainer.evaluate(full_context_dataset)])
df_context_results = pd.DataFrame([context_trainer.evaluate(context_dataset)])
df_no_context_results = pd.DataFrame([no_context_trainer.evaluate(no_context_dataset)])


In [14]:
df_results = pd.concat([df_no_context_results, df_context_results, df_full_context_results])
df_results["index"] = ["No context", "Title", "Title and Body"]
df_results.set_index("index")

,eval_loss,eval_accuracy,eval_f1,eval_macro_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,init_mem_cpu_alloc_delta,init_mem_gpu_alloc_delta,init_mem_cpu_peaked_delta,init_mem_gpu_peaked_delta,eval_mem_cpu_alloc_delta,eval_mem_gpu_alloc_delta,eval_mem_cpu_peaked_delta,eval_mem_gpu_peaked_delta
index,,,,,,,,,,,,,,,,
No context,0.65833,0.88945,0.63525,0.78505,0.66545,0.60768,43.38140,261.47100,44406,0,18258,0,232663,0,813237,138756096
Title,0.76379,0.90585,0.66245,0.80387,0.76664,0.58319,89.47930,126.76700,44474,0,18258,0,234701,0,813973,377946112
Title and Body,0.86059,0.85938,0.25153,0.58696,0.80240,0.14914,196.87390,57.61600,44294,0,18258,0,249674,0,812953,1158316032


## Dev results

In [8]:
df_results = pd.concat([df_no_context_results, df_context_results, df_full_context_results])
df_results["index"] = ["No context", "Title", "Title and Body"]
df_results.set_index("index")

,eval_loss,eval_accuracy,eval_f1,eval_macro_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,init_mem_cpu_alloc_delta,init_mem_gpu_alloc_delta,init_mem_cpu_peaked_delta,init_mem_gpu_peaked_delta,eval_mem_cpu_alloc_delta,eval_mem_gpu_alloc_delta,eval_mem_cpu_peaked_delta,eval_mem_gpu_peaked_delta
index,,,,,,,,,,,,,,,,
No context,0.55797,0.90655,0.67655,0.81097,0.71543,0.64167,35.27390,258.15100,43532,0,18258,0,196438,0,690153,138711040
Title,0.59812,0.92335,0.73174,0.84351,0.78354,0.68637,71.85510,126.72700,52658,0,18258,0,201723,0,689689,377901056
Title and Body,0.81184,0.86448,0.26722,0.59628,0.75758,0.16222,155.88470,58.41500,43397,0,18258,0,378836,0,674653,1158270976
